In [55]:
import boto3
from datetime import datetime

logs = boto3.client('logs')

In [84]:
start = int(datetime(2020, 6, 20).timestamp()) * 1000
end = int(datetime.now().timestamp()) * 1000

In [85]:
start

1592611200000

In [94]:
response = logs.filter_log_events(
    logGroupName='/aws/lambda/guessInmate',
    startTime = start,
    endTime = end,
)

In [95]:
events = response['events']

In [97]:
while True:
    response = logs.filter_log_events(
        logGroupName='/aws/lambda/guessInmate',
        startTime = start,
        endTime = end,
        nextToken=response['nextToken']
    )
    
    if 'nextToken' not in response:
        break
    else:
        events += response['events']

In [108]:
with open('./events.p', 'wb') as fp:
    pickle.dump(events, fp)

## get all the failures

In [1]:
import pickle
import requests

In [2]:
with open('./events.p', 'rb') as fp:
    logs = pickle.load(fp)

In [3]:
fails = list(
    filter(
        lambda s: 'FAIL' in s, [l['message'] for l in logs]))

In [4]:
ids = list(map(lambda s: s.split(':')[-1].split('\n')[0].strip(), fails))

In [5]:
endpoint = 'https://cwuo75bqj2.execute-api.us-east-2.amazonaws.com/guessInmate'

In [6]:
ids[0]

'02400-080'

In [7]:
import random
import string
import json

def get_random_alphanumeric_string(stringLength=8):
    lettersAndDigits = string.ascii_letters + string.digits
    return ''.join((random.choice(lettersAndDigits) for i in range(stringLength)))

In [8]:
def createSingleRequest(_id):
    inmateMin = _id.split('-')[0] # actual num
    regionNum = _id.split('-')[1] # actual region

    # get the range
    inmateMin = int(inmateMin) + 100000
    inmateMax = int(inmateMin) + 100001
    regionNum = int(regionNum) + 1000
    
    # pad the strings
    paddedInmateMin = "{}".format(inmateMin)
    paddedInmateMax = "{}".format(inmateMax)
    paddedRegionNum = "{}".format(regionNum)

    
    params = {
        "Id": get_random_alphanumeric_string(12),
        "MessageBody" : json.dumps({
            "inmateMin": paddedInmateMin,
            "inmateMax": paddedInmateMax,
            "regionNum": paddedRegionNum
        })
    }

    return params

In [9]:
batches = [createSingleRequest(_id) for _id in ids]

In [10]:
import boto3
sqs = boto3.client('sqs')
dir(sqs)

queueUrl = "https://us-east-2.queue.amazonaws.com/712505641269/inmateBatching"

In [13]:
sqs.send_message_batch(QueueUrl=queueUrl, Entries=[batches[0]])

{'Successful': [{'Id': 'Z6zwrf2bSEaP',
   'MessageId': 'ff6d6a86-c531-4aed-8057-8a17c032b37a',
   'MD5OfMessageBody': 'd80373b3cabd78de78a3edc8056bb845'}],
 'ResponseMetadata': {'RequestId': '6f5463e2-703f-5454-84be-349f81fdac03',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6f5463e2-703f-5454-84be-349f81fdac03',
   'date': 'Tue, 07 Jul 2020 03:44:02 GMT',
   'content-type': 'text/xml',
   'content-length': '478'},
  'RetryAttempts': 0}}

In [14]:
for i in range(1, len(batches), 10):
    sqs.send_message_batch(QueueUrl=queueUrl, Entries=batches[i:i+10])